In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, Concatenate, Dropout
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

AttributeError: module 'numpy' has no attribute 'integer'

In [ ]:
# Load images and wind data
def load_images_from_directory(directory, target_size=(512, 512)):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(directory, filename)
            img = load_img(img_path, target_size=target_size, color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
    return np.array(images)

# Load masked images
masked_images = load_images_from_directory('MaskedImages/')
masked_images = masked_images.astype('float32') / 255.0

# Create image pairs
x_train1 = np.array([masked_images[i] for i in range(len(masked_images)-1)])
x_train2 = np.array([masked_images[i+1] for i in range(len(masked_images)-1)])

# Load and prepare wind data
x_wind_array = np.load('arrow_info_array.npy')
x_wind_array = tf.image.resize(x_wind_array, (512, 512))
x_wind_array = tf.image.rgb_to_grayscale(x_wind_array)
x_train_wind = np.tile(x_wind_array, (len(x_train1), 1, 1, 1))

In [ ]:
# Model definition
input_img1 = Input(shape=(512, 512, 1))
input_img2 = Input(shape=(512, 512, 1))
input_wind = Input(shape=(512, 512, 1))

def encoder_block(input_layer):
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
    x = Dropout(0.2)(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    return x

encoded1 = encoder_block(input_img1)
encoded2 = encoder_block(input_img2)
encoded_wind = encoder_block(input_wind)

combined_encoded = Concatenate()([encoded1, encoded2, encoded_wind])

def decoder_block(input_layer):
    x = Conv2DTranspose(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = Dropout(0.2)(x)
    x = Conv2DTranspose(16, (3, 3), activation='relu', padding='same')(x)
    return x

x = decoder_block(combined_encoded)
decoded = Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model([input_img1, input_img2, input_wind], decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
# Train the model
autoencoder.fit([x_train1, x_train2, x_train_wind], x_train1, 
                epochs=50, batch_size=16, validation_split=0.2)

In [1]:
# Generate and visualize results
generated_images = autoencoder.predict([x_train1, x_train2, x_train_wind])

# Select sample indices to visualize
sample_indices = [0, 5, 10]  # Change these as needed

plt.figure(figsize=(20, 10))
for i, idx in enumerate(sample_indices):
    # Input image 1
    plt.subplot(3, 4, i*4 + 1)
    plt.imshow(x_train1[idx].squeeze(), cmap='gray')
    plt.title(f"Input 1 (Frame {idx})")
    plt.axis('off')
    
    # Generated image
    plt.subplot(3, 4, i*4 + 2)
    plt.imshow(generated_images[idx].squeeze(), cmap='gray')
    plt.title(f"Generated Frame {idx+1}")
    plt.axis('off')
    
    # Actual middle frame (if available)
    plt.subplot(3, 4, i*4 + 3)
    try:
        actual_frame = x_train2[idx].squeeze()
        plt.imshow(actual_frame, cmap='gray')
        plt.title(f"Actual Frame {idx+1}")
    except:
        plt.text(0.5, 0.5, 'No ground truth available', ha='center')
    plt.axis('off')
    
    # Input image 2
    plt.subplot(3, 4, i*4 + 4)
    plt.imshow(x_train2[idx].squeeze(), cmap='gray')
    plt.title(f"Input 2 (Frame {idx+1})")
    plt.axis('off')

plt.tight_layout()
plt.show()

NameError: name 'autoencoder' is not defined

In [ ]:
# Save generated images
os.makedirs('GeneratedImages/', exist_ok=True)
for i, img in enumerate(generated_images):
    plt.imsave(f'GeneratedImages/generated_image_{i+1}.png', img.squeeze(), cmap='gray')